<a href="https://colab.research.google.com/github/uadyf/Bootcamp-2021/blob/main/tugas_akhir/scrape_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [ ]:
ACCESS = 'ACCESS'
ACCESS_S = 'ACCESS_S'
TOKEN = 'TOKEN'
TOKEN_S = 'TOKEN_S'

In [ ]:
auth = tweepy.OAuthHandler(ACCESS, ACCESS_S)
auth.set_access_token(TOKEN, TOKEN_S)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
def filterTweet(tweet, key_outside, key_inside=None):
  if key_inside is not None:
    result1 = tweet._json[str(key_outside)][str(key_inside)]
    return result1
  else:
    result2 = tweet._json[str(key_outside)]
    return result2

def searchTweet(query, since, until, items):
  inside_user = ['id', 'name', 'screen_name', 'location', 'created_at', 'followers_count', 'friends_count', 'verified']
  inside_tweet = ['id', 'created_at', 'full_text']

  user_out = []
  tweet_out = []
  for tweet in tweepy.Cursor(api.search, q=query, since=since, until=until, tweet_mode='extended').items(items):
    out_user = []
    out_tweet = []
    
    for col_user in inside_user:
      result_user = filterTweet(tweet, 'user', col_user)
      out_user.append(result_user)
    
    for tw in inside_tweet:
      result_tw = filterTweet(tweet, tw)
      out_tweet.append(result_tw)
      
    user_out.append(out_user)
    tweet_out.append(out_tweet)
  final_user = replaceToNan(user_out)
  final_tweet = replaceToNan(tweet_out)
  return final_user, final_tweet

def replaceToNan(list_):
  replaced = []
  for i in range(len(list_)):
    new = [np.nan if val=='' else val for val in list_[i]]
    replaced.append(new)
  return replaced

def loop(query):
  SINCE = '2021-01-28'
  UNTIL = '2021-02-05'
  ITEMS = 1000
  
  out_user = []
  out_tweet = []
  for i in query:
    user_all = []
    tweeted = []
    user, tweet = searchTweet(i, SINCE, UNTIL, ITEMS)
    user_all.append(user)
    tweeted.append(tweet)

    out_user.append(user_all)
    out_tweet.append(tweeted)
  return out_user, out_tweet

def con(path):
  try:
    con = sqlite3.connect(str(path))
    return con
  except Error:
    print(Error)

In [ ]:
q = ['Rachel+Vennya', '#Selingkuh']
out_user, out_tweet = loop(q)

In [ ]:
inside_user = ['id', 'name', 'screen_name', 'location', 'created_at', 'followers_count', 'friends_count', 'verified']
inside_tweet = ['id', 'created_at', 'full_text']
df_user = pd.DataFrame(out_user[0][0]+out_user[1][0], columns=inside_user)
df_tweet = pd.DataFrame(out_tweet[0][0]+out_tweet[1][0], columns=inside_tweet)

In [ ]:
df_user

,id,name,screen_name,location,created_at,followers_count,friends_count,verified
0,892670807121330177,Ferdy Agusman,gusmanerdy,Medan,Wed Aug 02 08:58:00 +0000 2017,6,14,False
1,277434635,Karina angelika,karigel13,between his lungs,Tue Apr 05 11:03:15 +0000 2011,417,378,False
2,544170238,Kanjeng Sultan,ayiekkresna,"Kota Surabaya, Jawa Timur",Tue Apr 03 10:42:42 +0000 2012,590,333,False
3,1356180001888976899,UrExtacy,6hndrd6ty6,Bikini Bottom,Mon Feb 01 09:58:22 +0000 2021,2,114,False
4,1168873359741874176,bela,twingkerbelll,NaN,Tue Sep 03 13:08:38 +0000 2019,42,48,False
...,...,...,...,...,...,...,...,...
1995,1356663513024880641,Dania Putri,DaniaPu19568142,NaN,Tue Feb 02 17:59:41 +0000 2021,5,0,False
1996,1270945722259529728,AL,rohfakboi,NaN,Thu Jun 11 05:07:38 +0000 2020,34,105,False
1997,1291683283516850178,Rexsy Aprino Sanggen,rexsy_aprino,"Central Borneo, Indonesia",Fri Aug 07 10:31:18 +0000 2020,5,5,False
1998,1011527885876654080,pijat panggilan,SpaPanggilan,"surabaya, malang, palembang,",Tue Jun 26 08:33:36 +0000 2018,2072,222,False


In [ ]:
df_user.to_sql('userr', con('/content/Basisdata(raw).db'))

In [ ]:
df_tweet

,id,created_at,full_text
0,1357479018472239104,Thu Feb 04 23:59:53 +0000 2021,"RT @mhmdadipati: @farisandani Dari orang"" yang..."
1,1357478833725845507,Thu Feb 04 23:59:09 +0000 2021,RT @iirman_: Dari Rachel Vennya jd inget kata ...
2,1357478819456720896,Thu Feb 04 23:59:05 +0000 2021,RT @handokotjung: Berhasil membuat banyak neti...
3,1357478818701733888,Thu Feb 04 23:59:05 +0000 2021,RT @yeontinnie: dari rachel vennya kita belaja...
4,1357478707884032001,Thu Feb 04 23:58:39 +0000 2021,RT @mncveronica: Cerita Rachel Vennya dan Okin...
...,...,...,...
1995,1357367016533880832,Thu Feb 04 16:34:50 +0000 2021,VCS brbayar😘\nNew akun 💦\nWa:083191018873 \n#v...
1996,1357366907872047104,Thu Feb 04 16:34:24 +0000 2021,"gapapa wajar lagi musim, apalagi musim ujan gi..."
1997,1357366617651306496,Thu Feb 04 16:33:14 +0000 2021,"Selingkuh memang salah sih, tapi mungkin itu b..."
1998,1357366613327052800,Thu Feb 04 16:33:13 +0000 2021,RT @SpaPanggilan: #Selingkuh #sange cek video ...


In [ ]:
df_tweet.to_sql('tweeet', con('/content/Basisdata(raw).db'))